## Part 2
Dijkstra’s and Bellman Ford’s are single source shortest path algorithms. However, many times we are
faced with problems that require us to solve shortest path between all pairs. This means that the algorithm
needs to find the shortest path from every possible source to every possible destination. For every pair of
vertices u and v, we want to compute shortest path 𝑑𝑖𝑠𝑡𝑎𝑛𝑐𝑒(𝑢, 𝑣) and the second-to-last vertex on the
shortest path 𝑝𝑟𝑒𝑣𝑖𝑜𝑢𝑠(𝑢, 𝑣). How would you design an all-pair shortest path algorithm for both positive
edge weights and negative edge weights? Implement a function that can address this.


In [62]:
import random

In [63]:
# weighted digraph 
class DWG:

    def __init__(self):
        self.adj = {}
        self.weights = {}

    def are_connected(self, node1, node2):
        for neighbour in self.adj[node1]:
            if neighbour == node2:
                return True
        return False

    def adjacent_nodes(self, node):
        return self.adj[node]

    def add_node(self, node):
        self.adj[node] = []

    def add_edge(self, node1, node2, weight):
        if node2 not in self.adj[node1]:
            self.adj[node1].append(node2)
        self.weights[(node1, node2)] = weight

    def w(self, node1, node2):
        if self.are_connected(node1, node2):
            return self.weights[(node1, node2)]

    def num_nodes(self):
        return len(self.adj)

In [68]:
# min pq necessary for djisktra's 
class Item:
    def __init__(self, value, key):
        self.value = value
        self.key = key

class MinHeap:
    def __init__(self, elements):
        self.heap = elements
        self.positions = {element.value: i for i, element in enumerate(elements)}
        self.size = len(elements)
        self.build_heap()

    def parent(self, i):
        return (i - 1) // 2

    def left(self, i):
        return 2 * i + 1

    def right(self, i):
        return 2 * i + 2

    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        self.positions[self.heap[i].value], self.positions[self.heap[j].value] = i, j

    def min_heapify(self, i):
        l = self.left(i)
        r = self.right(i)
        smallest = i
        if l < self.size and self.heap[l].key < self.heap[i].key:
            smallest = l
        if r < self.size and self.heap[r].key < self.heap[smallest].key:
            smallest = r
        if smallest != i:
            self.swap(i, smallest)
            self.min_heapify(smallest)

    def build_heap(self):
        for i in range(self.size // 2, -1, -1):
            self.min_heapify(i)

    def extract_min(self):
        min_element = self.heap[0]
        self.size -= 1
        self.heap[0] = self.heap[self.size]
        self.positions[self.heap[0].value] = 0
        self.heap.pop()
        self.min_heapify(0)
        return min_element

    def decrease_key(self, value, new_key):
        i = self.positions[value]
        if new_key < self.heap[i].key:
            self.heap[i].key = new_key
            while i > 0 and self.heap[self.parent(i)].key > self.heap[i].key:
                self.swap(i, self.parent(i))
                i = self.parent(i)

    def insert(self, element):
        self.size += 1
        self.heap.append(element)
        self.positions[element.value] = self.size - 1
        self.decrease_key(element.value, element.key)

    def is_empty(self):
        return self.size == 0

In [89]:
def dijkstra_all_pairs(graph):
    all_shortest_paths = {}
    all_prev_nodes = {}
    for src in graph.adj.keys():
        dist = {}
        prev = {}
        heap_elements = [Item(vertex, float('inf')) for vertex in graph.adj.keys()]
        min_heap = MinHeap(heap_elements)
        min_heap.decrease_key(src, 0)
        while not min_heap.is_empty():
            current_element = min_heap.extract_min()
            u = current_element.value
            dist[u] = current_element.key
            for v in graph.adjacent_nodes(u):
                weight = graph.w(u, v)
                if weight is not None:
                    if v not in dist:
                        min_heap.decrease_key(v, dist[u] + weight)
                        prev[v] = u  # Store previous node
        all_shortest_paths[src] = {vertex: dist[vertex] for vertex in graph.adj.keys()}
        all_prev_nodes[src] = {vertex: prev[vertex] if vertex in prev else None for vertex in graph.adj.keys()}
    return all_shortest_paths, all_prev_nodes


In [93]:
def bellman_ford_all_pairs(g):
    all_shortest_paths = {}
    all_prev_nodes = {}
    for src in g.adj.keys():
        dist = {}
        paths = {src: [src]}
        nodes = list(g.adj.keys())
        relax_count = {}
        prev = {}

        for i in g.adj.keys():
            relax_count[i] = 0

        for node in nodes:
            dist[node] = float("inf")

        dist[src] = 0

        for _ in range(g.num_nodes() - 1):
            for node in nodes:
                for neighbour in g.adj[node]:
                    if relax_count[neighbour] < g.num_nodes() - 1 and dist[neighbour] > dist[node] + g.w(node, neighbour):
                        dist[neighbour] = dist[node] + g.w(node, neighbour)
                        prev[neighbour] = node  # Store previous node
                        relax_count[neighbour] += 1

        all_shortest_paths[src] = {vertex: dist[vertex] for vertex in g.adj.keys()}
        all_prev_nodes[src] = {vertex: prev[vertex] if vertex in prev else None for vertex in g.adj.keys()}

    return all_shortest_paths, all_prev_nodes


In [94]:
g = DWG() 
g.add_node(0)
g.add_node(1)
g.add_node(2)
g.add_node(3) 
g.add_edge(0, 1, 4)
g.add_edge(0, 2, 3)
g.add_edge(1, 2, 1)
g.add_edge(1, 3, 2)
g.add_edge(2, 3, 5)
#print(g.adj())
print("All pairs shortest paths with positive weights:")
print(dijkstra_all_pairs(g))
# Define the graph
g1 = DWG()
g1.add_node(0)
g1.add_node(1)
g1.add_node(2)
g1.add_node(3) 
g1.add_node(4)
g1.add_edge(0, 1, -1)
g1.add_edge(0, 2, 4)
g1.add_edge(1, 2, 3)
g1.add_edge(1, 3, 2)
g1.add_edge(1, 4, 2)
g1.add_edge(3, 2, 5)
g1.add_edge(3, 1, 1)
g1.add_edge(4, 3, -3)

# Test the Bellman-Ford algorithm
output = bellman_ford_all_pairs(g1)
print(output)


All pairs shortest paths with positive weights:
({0: {0: 0, 1: 4, 2: 3, 3: 6}, 1: {0: inf, 1: 0, 2: 1, 3: 2}, 2: {0: inf, 1: inf, 2: 0, 3: 5}, 3: {0: inf, 1: inf, 2: inf, 3: 0}}, {0: {0: None, 1: 0, 2: 0, 3: 1}, 1: {0: None, 1: None, 2: 1, 3: 2}, 2: {0: None, 1: 0, 2: None, 3: 2}, 3: {0: None, 1: None, 2: 1, 3: None}})
({0: {0: 0, 1: -1, 2: 2, 3: -2, 4: 1}, 1: {0: inf, 1: 0, 2: 3, 3: -1, 4: 2}, 2: {0: inf, 1: inf, 2: 0, 3: inf, 4: inf}, 3: {0: inf, 1: 1, 2: 4, 3: 0, 4: 3}, 4: {0: inf, 1: -2, 2: 1, 3: -3, 4: 0}}, {0: {0: None, 1: 0, 2: 1, 3: 4, 4: 1}, 1: {0: None, 1: None, 2: 1, 3: 4, 4: 1}, 2: {0: None, 1: None, 2: None, 3: None, 4: None}, 3: {0: None, 1: 3, 2: 1, 3: None, 4: 1}, 4: {0: None, 1: 3, 2: 1, 3: 4, 4: None}})
